# Урок 3. Построение надежных схем валидации решения, оптимизация целевых метрик

Основное задание:  
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.  
Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.  
Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.  
Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.  
​Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.  
​Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.  
​Задание 6 (опциональное): сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.  
​Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

In [ ]:
from tqdm import tqdm
from typing import List, Tuple
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GroupKFold
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from scipy.stats import ttest_rel
import warnings
warnings.simplefilter("ignore")

In [ ]:
!unzip /content/drive/MyDrive/assignment2_data.zip

Archive:  /content/drive/MyDrive/assignment2_data.zip
  inflating: assignment_2_test.csv   
  inflating: assignment_2_train.csv  


In [ ]:
data = pd.read_csv('/content/assignment_2_train.csv')
lb = pd.read_csv('/content/assignment_2_test.csv')

In [ ]:
num_features = data.select_dtypes(exclude=[np.object]).columns.to_list()
data_num = data[num_features]
lb_num = lb[num_features]

In [ ]:
X_data = data_num.drop('isFraud', axis=1)
y_data = data_num['isFraud']
X_lb = lb_num.drop('isFraud', axis=1)
y_lb = lb_num['isFraud']

# Задание 1

In [ ]:
X_train, X_valid = train_test_split(X_data, train_size=0.7, shuffle=True, random_state=5)
y_train, y_valid = train_test_split(y_data, train_size=0.7, shuffle=True, random_state=5)
X_train.shape, y_train.shape,  X_valid.shape, y_valid.shape

((125999, 379), (125999,), (54001, 379), (54001,))

In [ ]:
model = xgb.XGBClassifier(random_state=5)

model.fit(X=X_train, y=y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=20,
            eval_metric="auc",
            verbose=20)

[0]	validation_0-auc:0.633494	validation_1-auc:0.624628
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[20]	validation_0-auc:0.817529	validation_1-auc:0.813471
[40]	validation_0-auc:0.870684	validation_1-auc:0.867461
[60]	validation_0-auc:0.886423	validation_1-auc:0.880535
[80]	validation_0-auc:0.895336	validation_1-auc:0.887068
[99]	validation_0-auc:0.900372	validation_1-auc:0.891036


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=5,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
r1=roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:])
r2=roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:])
r3=roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1:])

In [ ]:
print (f'roc_auc_score: {r1.round(3)}, {r2.round(3)}, {r3.round(3)} ')

roc_auc_score: 0.9, 0.891, 0.857 


Вывод:
- наблючается переобучение
- между метриками валидационной выборки и выборки ЛБ существенный разрыв
- Hold-Out валидация с разбиением на 2 выборки не является устойчивой

# Задание 2

In [ ]:
X_train, X_valid = train_test_split(X_data, train_size=0.6, shuffle=True, random_state=5)
y_train, y_valid = train_test_split(y_data, train_size=0.6, shuffle=True, random_state=5)
X_valid, X_test = train_test_split(X_valid, train_size=0.5, shuffle=True, random_state=27)
y_valid, y_test = train_test_split(y_valid, train_size=0.5, shuffle=True, random_state=27)
X_train.shape, X_valid.shape, X_test.shape

((108000, 379), (36000, 379), (36000, 379))

In [ ]:
model = xgb.XGBClassifier(random_state=5)
model.fit(X=X_train, y=y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=20,
            eval_metric="auc",
            verbose=20)

[0]	validation_0-auc:0.636519	validation_1-auc:0.622589
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[20]	validation_0-auc:0.818047	validation_1-auc:0.815439
[40]	validation_0-auc:0.871642	validation_1-auc:0.863279
[60]	validation_0-auc:0.886373	validation_1-auc:0.875389
[80]	validation_0-auc:0.89595	validation_1-auc:0.882264
[99]	validation_0-auc:0.900913	validation_1-auc:0.886154


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=5,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
r1=roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:])
r2=roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:])
r3=roc_auc_score(y_test, model.predict_proba(X_test)[: ,1:])
r4=roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1:])
print (f'roc_auc_score: {r1.round(3)}, {r2.round(3)}, {r3.round(3)},{r4.round(3)}  ')

roc_auc_score: 0.901, 0.886, 0.891,0.856  



Выводы те же, что и в задании 1

# Задание 3

In [ ]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [ ]:
np.random.seed(27)
scores = create_bootstrap_metrics(y_lb, model.predict_proba(X_lb)[:, 1:], roc_auc_score)
calculate_confidence_interval(scores)

(0.8489801142673735, 0.8625927851804445)

- Hold-Out валидация на 3 выборках устойчива

# Задание 4

In [ ]:
X_adv = pd.concat([X_data, X_lb], axis=0)
y_adv = np.hstack((np.zeros(X_data.shape[0]), np.ones(X_lb.shape[0])))
assert X_adv.shape[0] == y_adv.shape[0]

In [ ]:
model = xgb.XGBClassifier(n_estimators=25)
model.fit(X_adv, y_adv)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred_adv = model.predict_proba(X_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 4))

1.0


In [ ]:
y_pred = model.predict_proba(X_data)
y_pred

array([[0.96031356, 0.03968643],
       [0.96031356, 0.03968643],
       [0.96031356, 0.03968643],
       ...,
       [0.96031356, 0.03968643],
       [0.96031356, 0.03968643],
       [0.96031356, 0.03968643]], dtype=float32)

In [ ]:
pd.cut(y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)).value_counts().sort_index()

(0.0, 0.1]    180000
(0.1, 0.2]         0
(0.2, 0.3]         0
(0.3, 0.4]         0
(0.4, 0.5]         0
(0.5, 0.6]         0
(0.6, 0.7]         0
(0.7, 0.8]         0
(0.8, 0.9]         0
(0.9, 1.0]         0
dtype: int64

- все объекты обучаемой выборки похожи на объекты валидационной выборки
- можно использовать для обучения модели.

# Задание 5

In [ ]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [ ]:
cv_strategy = KFold(n_splits=5, random_state=1)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    X_data, y_data, model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.607, valid-score = 0.5547
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.6125, valid-score = 0.5865
Fold: 3, train-observations = 144000, valid-observations = 36000
train-score = 0.6048, valid-score = 0.614
Fold: 4, train-observations = 144000, valid-observations = 36000
train-score = 0.5971, valid-score = 0.5994
Fold: 5, train-observations = 144000, valid-observations = 36000
train-score = 0.593, valid-score = 0.59
CV-results train: 0.6029 +/- 0.007
CV-results valid: 0.5889 +/- 0.02
OOF-score = 0.5878


In [ ]:
cv_strategy = KFold(n_splits=5, random_state=1)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    X_lb, y_lb, model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 80000, valid-observations = 20001
train-score = 0.6136, valid-score = 0.596
Fold: 2, train-observations = 80001, valid-observations = 20000
train-score = 0.6116, valid-score = 0.5867
Fold: 3, train-observations = 80001, valid-observations = 20000
train-score = 0.6113, valid-score = 0.5666
Fold: 4, train-observations = 80001, valid-observations = 20000
train-score = 0.6076, valid-score = 0.6172
Fold: 5, train-observations = 80001, valid-observations = 20000
train-score = 0.6051, valid-score = 0.6143
CV-results train: 0.6098 +/- 0.003
CV-results valid: 0.5961 +/- 0.019
OOF-score = 0.5984


- KFold валидация является устойчивой

# Задание 6

In [ ]:
cv = TimeSeriesSplit()
for train_index, test_index in cv.split(X_data):
    X_train, X_valid = X_data.loc[train_index], X_data.loc[test_index]
    y_train, y_valid = y_data[train_index], y_data[test_index]   
X_train.shape,  X_valid.shape, y_train.shape, y_valid.shape

((150000, 379), (30000, 379), (150000,), (30000,))

In [ ]:
model = xgb.XGBClassifier(random_state=5)
model.fit(X=X_train, y=y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=20,
            eval_metric="auc",
            verbose=20)

[0]	validation_0-auc:0.638528	validation_1-auc:0.628491
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[20]	validation_0-auc:0.817302	validation_1-auc:0.809263
[40]	validation_0-auc:0.869654	validation_1-auc:0.84572
[60]	validation_0-auc:0.886744	validation_1-auc:0.859966
[80]	validation_0-auc:0.894275	validation_1-auc:0.865203
[99]	validation_0-auc:0.899564	validation_1-auc:0.86813


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=5,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
r1=roc_auc_score(y_train, model.predict_proba(X_train)[:, 1:])
r2=roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1:])
r3=roc_auc_score(y_lb, model.predict_proba(X_lb)[:, 1:])
print (f'roc_auc_score: {r1.round(3)}, {r2.round(3)}, {r3.round(3)} ')

roc_auc_score: 0.9, 0.868, 0.854 


In [ ]:
np.random.seed(27)
scores = create_bootstrap_metrics(y_lb, model.predict_proba(X_lb)[:, 1:], roc_auc_score)
calculate_confidence_interval(scores)

(0.8468448419251747, 0.860623234697544)

- присутствует существенный разрыв между метриками обучающей и ЛБ выборки, что говорит о неустойчивости.

# Задание 7

In [ ]:
def get_clientID(data):
    
    for n, k in enumerate(data['ProductCD'].value_counts().keys()):
        data.loc[data['ProductCD'] == k, 'ProdCD'] = n
    data['ProdCD'] = data['ProdCD'].astype(np.int8)

    data.loc[data['P_emaildomain'].isnull(), 'P_emaildomain'] = 'nan'
    for n, k in enumerate(data['P_emaildomain'].value_counts().keys()):
        data.loc[data['P_emaildomain'] == k, 'P_email'] = n
    data['P_email'] = data['P_email'].astype(np.int8)

    data['ClientID'] = data['ProdCD'].astype(str) + data['card1'].astype(str) + data['P_email'].astype(str)
    data['ClientID'] = data['ClientID'].astype(np.int32)

    data = data.drop(['ProdCD', 'P_email'], axis=1)
    
    num_features = data.select_dtypes(exclude=[np.object]).columns.to_list()
    data_num = data[num_features]
    
    X_data = data_num.drop('isFraud', axis=1)
    y_data = data_num['isFraud']
    
    return X_data, y_data

In [ ]:
X_data, y_data = get_clientID(data)
X_lb, y_lb = get_clientID(lb)

In [ ]:
def make_cross_validation_groups(X: pd.DataFrame,
                              y: pd.Series,
                              estimator: object,
                              metric: callable,
                              cv_strategy,
                              groups):

    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y, groups=X[groups])):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [ ]:
cv_strategy = GroupKFold(n_splits=5)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation_groups(
    X_data, y_data, model, metric=roc_auc_score, cv_strategy=cv_strategy, groups='ClientID')

Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.6616, valid-score = 0.6158
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.6536, valid-score = 0.638
Fold: 3, train-observations = 144000, valid-observations = 36000
train-score = 0.6621, valid-score = 0.6474
Fold: 4, train-observations = 144000, valid-observations = 36000
train-score = 0.6613, valid-score = 0.6169
Fold: 5, train-observations = 144000, valid-observations = 36000
train-score = 0.657, valid-score = 0.6214
CV-results train: 0.6591 +/- 0.003
CV-results valid: 0.6279 +/- 0.013
OOF-score = 0.629


In [ ]:
cv_strategy = GroupKFold(n_splits=5)
estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation_groups(
    X_lb, y_lb, model, metric=roc_auc_score, cv_strategy=cv_strategy, groups='ClientID')

Fold: 1, train-observations = 80000, valid-observations = 20001
train-score = 0.6585, valid-score = 0.6589
Fold: 2, train-observations = 80001, valid-observations = 20000
train-score = 0.6668, valid-score = 0.6106
Fold: 3, train-observations = 80001, valid-observations = 20000
train-score = 0.6604, valid-score = 0.6502
Fold: 4, train-observations = 80001, valid-observations = 20000
train-score = 0.6622, valid-score = 0.6249
Fold: 5, train-observations = 80001, valid-observations = 20000
train-score = 0.6676, valid-score = 0.6157
CV-results train: 0.6631 +/- 0.004
CV-results valid: 0.6321 +/- 0.019
OOF-score = 0.6332


- валидация GroupKFold по предположительному клиенту стабильна.